In [ ]:
from groq import Groq
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

client = Groq(api_key="")
index = faiss.read_index("faiss_index.bin")  
df = pd.read_excel("train_split.xlsx")
texts = df["text"].tolist()
labels = df["label"].tolist()
model = SentenceTransformer('all-MiniLM-L6-v2')


c:\Users\Mohamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def classify_text(query, k=3):
    query_emb = model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_emb, k)

    top_examples = [texts[i] for i in indices[0]]
    top_labels = [labels[i] for i in indices[0]]

    context = "\n".join([
        f"Example {i+1}:\nText: {top_examples[i]}\nLabel: {top_labels[i]}"
        for i in range(len(top_examples))
    ])

    prompt = f"""
You are a text classification assistant. 
Based on the following labeled examples, determine the most likely label for the new text.

Labeled examples:
{context}

Now classify this new text:
"{query}"

Respond only with the most appropriate label.
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    model_label = response.choices[0].message.content.strip()

    return model_label


In [ ]:
query_text = "اذا كنت تبحث عن ملابس جديده تحقق دايما من seasonal sales في fashion stores المحليه"
predicted_label = classify_text(query_text)
print("Predicted label:", predicted_label)


In [3]:
df = pd.read_excel("test_split.xlsx")
sample_df = df.sample(n=50, random_state=42).reset_index(drop=True)
texts_test = sample_df["text"].astype(str).tolist()
true_labels = sample_df["label"].astype(str).tolist()
predicted_labels = []
for i, query_text in enumerate(texts_test):
    predicted = classify_text(query_text)
    predicted_labels.append(predicted)


accuracy = accuracy_score(true_labels, predicted_labels)
print(f"{accuracy:.2f}")
print("Classification Report:")
print(classification_report(true_labels, predicted_labels))




Nearest neighbor labels: ['medical', 'medical', 'medical']
Groq model predicted label: medical
Nearest neighbor labels: ['shopping', 'shopping', 'shopping']
Groq model predicted label: shopping
Nearest neighbor labels: ['shopping', 'shopping', 'shopping']
Groq model predicted label: shopping
Nearest neighbor labels: ['shopping', 'shopping', 'shopping']
Groq model predicted label: shopping
Nearest neighbor labels: ['business', 'business', 'business']
Groq model predicted label: business
Nearest neighbor labels: ['education', 'education', 'education']
Groq model predicted label: education
Nearest neighbor labels: ['shopping', 'shopping', 'shopping']
Groq model predicted label: shopping
Nearest neighbor labels: ['finance', 'finance', 'finance']
Groq model predicted label: finance
Nearest neighbor labels: ['business', 'business', 'business']
Groq model predicted label: business
Nearest neighbor labels: ['finance', 'finance', 'finance']
Groq model predicted label: finance
Nearest neighbor l